In [1]:
# Dependencies
import pandas as pd
import numpy as np

# Create a reference the CSV file desired
json_path = "Resources/purchase_data.json"

# Read the file into a Pandas DataFrame
# 'records' : list like [{column -> value}, ... , {column -> value}]
pymoli_df = pd.read_json(json_path, orient='records')

# Print the first five rows of data to the screen
#pymoli_df.head()

In [2]:
#rename SN -> Player
pymoli_df = pymoli_df.rename(columns = {'SN':'Player'})
pymoli_df.head()

,Age,Gender,Item ID,Item Name,Price,Player
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


**Player Count**

* Total Number of Players

In [4]:
#setup result dataframe
results_df = pd.DataFrame(columns = ['Total Players'])

#calculate number of unique players
results_df.loc[1] = pymoli_df['Player'].nunique()

results_df

,Total Players
1,573


**Purchasing Analysis (Total)**

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [5]:
#calculate values
results_df['Number of Unique Items'] = pymoli_df['Item ID'].nunique()
results_df['Average Purchase Price'] = round(pymoli_df['Price'].mean(),2)
results_df['Total Number of Purchases'] = pymoli_df.shape[0]
results_df['Total Revenue'] = pymoli_df['Price'].sum()

#assign formats
results_df['Average Purchase Price'] = results_df['Average Purchase Price'].map('${:,.2f}'.format)
results_df['Total Revenue'] = results_df['Total Revenue'].map('${:,.2f}'.format)
results_df

,Total Players,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
1,573,183,$2.93,780,"$2,286.33"


#  Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [6]:
#setup result dataframe and calculate totals
demografics_df = (pymoli_df.groupby('Gender')['Player'].nunique()).to_frame()

#remove index name
demografics_df.index.name = ''

#calculate pecentage
demografics_df['Pecentage of Players'] = ( demografics_df['Player'] / \
                                           demografics_df['Player'].sum() * 100 \
                                           ).map('{0:.2f}%'.format)

#rename columns
demografics_df = demografics_df.rename(columns={'Player':'Total Count'})

#swap columns
demografics_df = demografics_df.reindex(columns = ['Pecentage of Players','Total Count'])

demografics_df

,Pecentage of Players,Total Count
,,
Female,17.45%,100
Male,81.15%,465
Other / Non-Disclosed,1.40%,8


**Purchasing Analysis (Gender)** 

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

In [7]:
#setup result dataframe and calculate totals
purchases_df = pymoli_df.groupby('Gender').aggregate({'Item ID':np.count_nonzero, 'Price':np.sum})
                
#remove index name
purchases_df.index.name = ''

#rename columns
purchases_df = purchases_df.rename(columns={'Item ID':'Purchase Count', 'Price':'Total Purchase Value'})

#calculate values
purchases_df['Average Purchase Price'] = (purchases_df['Total Purchase Value']/purchases_df['Purchase Count']) \
                                            .map('${:,.2f}'.format)

purchases_df['Normalized Totals'] = (purchases_df['Total Purchase Value']/demografics_df['Total Count']) \
                                            .map('${:,.2f}'.format)

purchases_df

,Purchase Count,Total Purchase Value,Average Purchase Price,Normalized Totals
,,,,
Female,136,382.91,$2.82,$3.83
Male,632,1867.68,$2.96,$4.02
Other / Non-Disclosed,11,35.74,$3.25,$4.47


**Age Demographics**

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

In [8]:
#assign bins
bins = [0,10,14,19,24,29,34,39,100]
bins_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

#add 'Age Group' column to original df
pymoli_df['Age Group'] = pd.cut(pymoli_df['Age'], bins, labels=bins_names)

#group players by age
players_age_df = (pymoli_df.groupby('Age Group')['Player'].nunique()).to_frame()

#remove index name
players_age_df.index.name = ''

#rename columns
players_age_df.columns = ['Total Count of Players']

#calculate percentage
players_age_df['Percentage of Players'] = ( players_age_df['Total Count of Players'] / \
                                            players_age_df['Total Count of Players'].sum() * 100 \
                                          ) .map('{0:.2f}%'.format)
                                          

players_age_df

,Total Count of Players,Percentage of Players
,,
<10,22,3.84%
10-14,20,3.49%
15-19,100,17.45%
20-24,259,45.20%
25-29,87,15.18%
30-34,47,8.20%
35-39,27,4.71%
40+,11,1.92%


In [14]:
#calculate purchase totals and counts
age_df = pymoli_df.groupby('Age Group').aggregate({'Item ID':np.count_nonzero, 'Price':np.sum})

#rename columns
age_df = age_df.rename(columns={'Item ID':'Purchase Count', 'Price':'Total Purchase Value'})

#calculate average purchase value
age_df['Average Purchase Value'] = (age_df['Total Purchase Value'] / age_df['Purchase Count']) \
                                    .map('${:,.2f}'.format)

#calculate normalized purchase value
age_df['Normalized Purchase Value'] = (age_df['Total Purchase Value'] / \
                                       players_age_df['Total Count of Players']).map('${:,.2f}'.format)

#calculate percent of all purchases
age_df['Percentage of Purchases'] = (age_df['Total Purchase Value'] / \
                                     age_df['Total Purchase Value'].sum() * 100 \
                                    ).map('{0:.2f}%'.format)

age_df

,Purchase Count,Total Purchase Value,Average Purchase Value,Normalized Purchase Value,Percentage of Purchases
Age Group,,,,,
<10,32,96.62,$3.02,$4.39,4.23%
10-14,31,83.79,$2.70,$4.19,3.66%
15-19,133,386.42,$2.91,$3.86,16.90%
20-24,336,978.77,$2.91,$3.78,42.81%
25-29,125,370.33,$2.96,$4.26,16.20%
30-34,63,197.25,$3.13,$4.20,8.63%
35-39,42,119.40,$2.84,$4.42,5.22%
40+,17,53.75,$3.16,$4.89,2.35%


**Top Spenders**

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

In [10]:
#setup result dataframe and calculate totals
top_df = pymoli_df.groupby('Player').aggregate({'Item ID':np.count_nonzero, 'Price':np.sum})

#leave 5 top spenders only
top_df = top_df.nlargest(5, 'Price')

#calculate averages
top_df['Average Purchase Price'] = top_df['Price'] / top_df['Item ID']

#rename columns 
top_df = top_df.rename(columns={'Item ID':'Purchase Count', 'Price':'Total Purchase Value'})

#swap columns
top_df = top_df.reindex(columns = ['Purchase Count','Average Purchase Price','Total Purchase Value'])

#format values
top_df = top_df.style.format({'Average Purchase Price': "${:,.2f}", 'Total Purchase Value': '${:,.2f}'})


top_df

,Purchase Count,Average Purchase Price,Total Purchase Value
Player,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


**Most Popular Items**

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [11]:
#group pymoli_df by 'Item ID' and 'Item name' counting total purchase count
#as count of 'Item ID' and total price as sum of 'Price'
#and leave 5 max values of 'Price' column only
item_top_df = pymoli_df.groupby(['Item ID', 'Item Name', 'Price']) \
              .aggregate({'Item ID':np.count_nonzero, 'Price':np.sum}) \
              .nlargest(5, 'Item ID')
    
#rename columns
item_top_df.columns = ['Purchase Count','Total Purchase Value']

#make 'Price' index - result of groupby - to column
#can do that only after renaming columns since
#aggregate function of groupby has 'Price' name too
item_top_df = item_top_df.reset_index('Price')

#rename Price column
item_top_df = item_top_df.rename(columns={'Price':'Item Price'})

#swap columns
item_top_df = item_top_df.reindex(columns = ['Purchase Count', 'Item Price','Total Purchase Value'])
 
#format values
item_top_df = item_top_df.style.format({'Item Price': '${:,.2f}', 'Total Purchase Value': '${:,.2f}'})

                             
item_top_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
13,Serenity,9,$1.49,$13.41
31,Trickster,9,$2.07,$18.63
34,Retribution Axe,9,$4.14,$37.26


**Most Profitable Items**

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [19]:
#group pymoli_df by 'Item ID' and 'Item name' counting total purchase count
#as count of 'Item ID' and total price as sum of 'Price'
#and leave 5 max values of 'Price' column only
item_top_df = pymoli_df.groupby(['Item ID', 'Item Name', 'Price']) \
              .aggregate({'Item ID':np.count_nonzero, 'Price':np.sum}) \
              .nlargest(5, 'Price')
    
#rename columns
item_top_df.columns = ['Purchase Count','Total Purchase Value']

#make 'Price' index - result of groupby - to column
#can do that only after renaming columns since
#aggregate function of groupby has 'Price' name too
item_top_df = item_top_df.reset_index('Price')

#rename Price column
item_top_df = item_top_df.rename(columns={'Price':'Item Price'})

#swap columns
item_top_df = item_top_df.reindex(columns = ['Purchase Count', 'Item Price','Total Purchase Value'])
 
#calculate total purchases
total_purchases = pymoli_df['Price'].sum()

#calculate percentage of total purcahses per item
item_top_df['Percentage of purchases'] = (item_top_df['Total Purchase Value'] / \
                                          total_purchases * 100 \
                                         ).map('{0:.2f}%'.format)    
    
#format values
item_top_df = item_top_df.style.format({'Item Price': '${:,.2f}', 'Total Purchase Value': '${:,.2f}'})

item_top_df

,,Purchase Count,Item Price,Total Purchase Value,Percentage of purchases
Item ID,Item Name,,,,
34,Retribution Axe,9,$4.14,$37.26,1.63%
115,Spectral Diamond Doomblade,7,$4.25,$29.75,1.30%
32,Orenmir,6,$4.95,$29.70,1.30%
103,Singed Scalpel,6,$4.87,$29.22,1.28%
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88,1.26%


## Observations:
* 80% of the players are male
* Almost half of the players are between 20-24 yo
* All age groups spend about the same amount of money per player
* Top purchased items are about the same percentage of all purchases